In [ ]:
!pip install pandas numpy scikit-learn imbalanced-learn


In [6]:
!pip install pandas
!pip install numpy
!pip install scikit-learn
!pip install imbalanced-learn


You should consider upgrading via the 'C:\Users\tjddl\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'C:\Users\tjddl\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'C:\Users\tjddl\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'C:\Users\tjddl\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


In [7]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OrdinalEncoder
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, VotingClassifier
from sklearn.feature_selection import SelectFromModel
from sklearn.ensemble import ExtraTreesClassifier
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import roc_auc_score

# 데이터 불러오기
train = pd.read_csv('./train.csv').drop(columns=['ID'])
test = pd.read_csv('./test.csv').drop(columns=['ID'])

X = train.drop('임신 성공 여부', axis=1)
y = train['임신 성공 여부']

# 카테고리형 변수 처리
categorical_columns = X.select_dtypes(include='object').columns
for col in categorical_columns:
    X[col] = X[col].astype(str)
    test[col] = test[col].astype(str)

encoder = OrdinalEncoder()
X[categorical_columns] = encoder.fit_transform(X[categorical_columns])
test[categorical_columns] = encoder.transform(test[categorical_columns])

# 특징 선택 (Extra Trees 기반)
feature_selector = ExtraTreesClassifier(n_estimators=100).fit(X, y)
model = SelectFromModel(feature_selector, prefit=True)
X_selected = model.transform(X)
test_selected = model.transform(test)

# 데이터 불균형 처리 (SMOTE 적용)
smote = SMOTE(random_state=42)
X_balanced, y_balanced = smote.fit_resample(X_selected, y)

# 데이터 분할
X_train, X_val, y_train, y_val = train_test_split(X_balanced, y_balanced, test_size=0.2, random_state=42)

# 앙상블 모델 구성
rf = RandomForestClassifier(n_estimators=100, max_depth=10, random_state=42)
ada = AdaBoostClassifier(n_estimators=100, random_state=42)

# 투표 기반 분류기 (Soft Voting)
voting_model = VotingClassifier(estimators=[('rf', rf), ('ada', ada)], voting='soft')

# 하이퍼파라미터 튜닝
param_grid = {
    'rf__n_estimators': [100, 200],
    'rf__max_depth': [10, 20],
    'ada__n_estimators': [50, 100]
}
grid_search = GridSearchCV(voting_model, param_grid, cv=3, scoring='roc_auc')
grid_search.fit(X_train, y_train)

# 검증 데이터 예측 및 성능 평가
best_model = grid_search.best_estimator_
y_pred = best_model.predict_proba(X_val)[:, 1]

print("최적 하이퍼파라미터:", grid_search.best_params_)
print("ROC AUC Score:", roc_auc_score(y_val, y_pred))

# 테스트 데이터 예측
test_predictions = best_model.predict_proba(test_selected)[:, 1]

# 결과 저장
submission = pd.DataFrame({
    'ID': test['ID'],
    'probability': test_predictions
})
submission.to_csv('submission.csv', index=False)


ModuleNotFoundError: No module named 'imblearn'